In [ ]:
import pickle
import math
import torch
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
data = pickle.load(open('quant_data_old.pkl', 'rb'))
type(data)

In [ ]:
for layer_name, layer_data in data.items():
    print(f'layer_name: {layer_name}')
    for data_name in ['output', 'weight']:
        if data_name in layer_data:
            scale_bit = math.log(layer_data[data_name]['scale'], 0.5)
            assert abs(scale_bit - round(scale_bit)) < 1e-6
            print(f"{data_name} scale_bit: {round(scale_bit)}, zero_point: {layer_data[data_name]['zero_point']}")

In [ ]:
inp = data['quant']['output']
inp_data = torch.tensor(inp['data'])

In [ ]:
kernel = data['conv.0.conv']['weight']
kernel_data = torch.tensor(kernel['data'])

In [ ]:
out = data['conv.0.conv']['output']

In [ ]:
o = F.conv2d((inp_data-64), kernel_data, stride=2, padding=0)
o = F.relu(o)

In [ ]:
torch.round(o / 2**10).numpy()

In [ ]:
out['data']

In [ ]:
data['fc.2']['output']

In [ ]:
plt.imshow(data['quant']['output']['data'][0].transpose([1,2,0]).astype(np.uint8))

In [19]:
## export some horse images
import os
os.chdir('../')
from PIL import Image
import torchvision


In [20]:
CIFAR_ROOT = '/media/HD1/Datasets/cifar'
trainset = torchvision.datasets.CIFAR10(
    root=CIFAR_ROOT, train=True, download=False)
print(trainset.classes)

['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']


In [21]:
out_dir = 'data/horse_samples'
os.makedirs(out_dir, exist_ok=True)
count = 0
for img, label in trainset:
    if label == trainset.class_to_idx['horse']:
        img.save(f'{out_dir}/{count:04d}.png')
        count += 1

['airplane',
 'automobile',
 'bird',
 'cat',
 'deer',
 'dog',
 'frog',
 'horse',
 'ship',
 'truck']